In [3]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from environment import SepsisEnv
from model import run_agent

OSError: [WinError 127] The specified procedure could not be found. Error loading "C:\ProgramData\Anaconda3\lib\site-packages\torch\lib\nvfuser_codegen.dll" or one of its dependencies.

In [ ]:

sepsis = pd.read_csv('data/MIMICtable.csv')

In [2]:
# Determining observation space variables

observation_space = ['gender', 'mechvent', 'age', 're_admission', 'Weight_kg', 'GCS', 'HR', 'SysBP', 'MeanBP', 
 'DiaBP', 'RR','Temp_C', 'FiO2_1', 'Potassium', 'Sodium', 
 'Chloride', 'Glucose', 'Magnesium', 'Calcium', 'Hb', 'WBC_count', 'Platelets_count',
 'PT', 'PTT', 'Arterial_pH', 'paO2', 
 'paCO2', 'Arterial_BE','HCO3', 'Arterial_lactate', 'PaO2_FiO2', 'SpO2', 'SGOT','Creatinine','BUN',
 'SGPT','INR', 'Total_bili']

In [3]:
# Prepare variables for RL
sepsis['dt'] = pd.to_datetime(sepsis['charttime'], unit = 's')
sepsis['SOFA_NEXT'] = sepsis.groupby('icustayid')['SOFA'].shift(-1)
sepsis['reward'] = -0.25 * np.tanh(sepsis['SOFA'] - 6) + -0.2 * (sepsis['SOFA_NEXT'] - sepsis['SOFA'])
sepsis['step'] = sepsis.groupby('icustayid').cumcount(ascending = True)
sepsis['is_end'] = sepsis.groupby('icustayid')['step'].transform(lambda x: (x == x.max()).astype(int))

NameError: name 'sepsis' is not defined

In [337]:
# Transform observation space
scaler = StandardScaler()
scaled_data = scaler.fit_transform(sepsis[observation_space])
data_tensor = torch.FloatTensor(scaled_data)

In [338]:
class Autoencoder(nn.Module):
    def __init__(self, obs_space, dim):
        super(Autoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(obs_space, dim),
            nn.Linear(dim, obs_space)
        )

    def forward(self, x):
        x = self.encoder(x)
        return x

In [339]:
# Initialize the autoencoder
input_size = data_tensor.shape[1]
hidden_size = 16
autoencoder = Autoencoder(input_size, hidden_size)
criterion = nn.MSELoss()
optimizer = optim.Adam(autoencoder.parameters(), lr=0.001)

num_epochs = 100
for epoch in range(num_epochs):
    outputs = autoencoder(data_tensor)
    loss = criterion(outputs, data_tensor)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

In [340]:
sepsis['state'] = sepsis.apply(lambda row: list(outputs.detach().numpy()[row.name]), axis=1)

In [ ]:
actions = ['max_dose_vaso', 'input_total']

In [356]:
env = SepsisEnv(sepsis)

In [ ]:
run_agent(env)

In [347]:
sepsis[['icustayid', 'reward', 'is_end', 'state', 'step']]

,icustayid,reward,is_end,state,step
0,11,0.209601,0,"[0.7355523, 0.020328823, 0.7207098, 0.59214455...",0
1,11,0.190399,0,"[0.8348446, 0.4144567, 0.62653625, 0.3550992, ...",1
2,11,-0.009601,0,"[0.8471354, 0.26626393, 0.6448276, 0.47399607,...",2
3,11,-0.000000,0,"[0.6508782, 0.40743545, 0.28347248, 0.362692, ...",3
4,11,-0.800000,0,"[0.69932824, 0.01118131, 0.37214866, 0.502551,...",4
...,...,...,...,...,...
233627,99995,0.249997,0,"[-0.69728565, -0.2890424, -0.69142675, -0.0250...",8
233628,99995,-0.550003,0,"[-0.7468528, -0.28700665, -0.78307116, -0.1198...",9
233629,99995,1.041007,0,"[-0.57181424, -0.29108945, -0.7078303, 0.07258...",10
233630,99995,0.249997,0,"[-0.450858, -0.28231165, -0.57654035, 0.139152...",11
